In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#default_exp classify_image

In [3]:
#export
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
from food.psql import *

# !nbdev_build_lib
import requests
from food.clipmodel import image2clip
from PIL import Image

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [10]:
#export
from food.milvus import collection
collection.load()

In [11]:
#export
def search_by_clip(clip, topk=5, query_expr='',search_params = {'ef':10000}): #32768
    search_params = {"metric_type": "IP",
                     "params": search_params}

    print(query_expr)
    results = collection.search(clip[None,], "clip", search_params, limit=topk, expr=query_expr, output_fields=["food_id"])
    results = results[0]
    return list(results.distances), results.ids

def get_metadata(ids):
    q = f"""select name from foods where id in {tuple(ids)}"""
    return pd.read_sql(q,engine)

In [22]:
url = 'https://www.foodfanatic.co.uk/wp-content/uploads/2020/01/food-fanatic-roasted-duck-preverada-scaled-2560x1280.jpg'

In [31]:
#export
def search_image(url=None,path=None):
    if url:
        response = requests.get(url, stream=True)
        image = Image.open(response.raw)
    elif path:
        image = Image.open(path)
    clip = image2clip(image)
    results = search_by_clip(clip.numpy())
    df = get_metadata(results[1])
    df['score'] = results[0]
    return df

In [32]:
search_image(url=url)

,name,score
0,Roast Partridge--Bread Sauce,0.302072
1,"Roast prairie chicken, apple sauce",0.295918
2,"Braised Goose, Stuffed with Chestnuts, Apple S...",0.295918
3,Roast Partridge--Bread Sauce,0.292836
4,"Roast prairie chicken, apple sauce",0.292836


In [5]:
!nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 00_tools.ipynb.
Converted 04_milvus.ipynb.
Converted 06_clip.ipynb.
Converted 0_template.ipynb.
Converted bot.ipynb.
Converted classfy_image.ipynb.
Converted inference.ipynb.
Converted milvus_update.ipynb.
